## Importing Libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

**1164. Product Price at a Given Date (Medium)**

**Table: Products**

| Column Name   | Type    |
|---------------|---------|
| product_id    | int     |
| new_price     | int     |
| change_date   | date    |

(product_id, change_date) is the primary key (combination of columns with unique values) of this table.
Each row of this table indicates that the price of some product was changed to a new price at some date.
 
**Write a solution to find the prices of all products on 2019-08-16. Assume the price of all products before any change is 10.**

Return the result table in any order.

The result format is in the following example.

**Example 1:**

**Input:**
**Products table:**

| product_id | new_price | change_date |
|------------|-----------|-------------|
| 1          | 20        | 2019-08-14  |
| 2          | 50        | 2019-08-14  |
| 1          | 30        | 2019-08-15  |
| 1          | 35        | 2019-08-16  |
| 2          | 65        | 2019-08-17  |
| 3          | 20        | 2019-08-18  |

**Output:**
| product_id | price |
|------------|-------|
| 2          | 50    |
| 1          | 35    |
| 3          | 10    |


In [0]:
products_data_1164 = [
    (1, 20, "2019-08-14"),
    (2, 50, "2019-08-14"),
    (1, 30, "2019-08-15"),
    (1, 35, "2019-08-16"),
    (2, 65, "2019-08-17"),
    (3, 20, "2019-08-18"),
]

products_columns_1164 = ["product_id", "new_price", "change_date"]
products_df_1164 = spark.createDataFrame(products_data_1164, products_columns_1164)
products_df_1164.show()

+----------+---------+-----------+
|product_id|new_price|change_date|
+----------+---------+-----------+
|         1|       20| 2019-08-14|
|         2|       50| 2019-08-14|
|         1|       30| 2019-08-15|
|         1|       35| 2019-08-16|
|         2|       65| 2019-08-17|
|         3|       20| 2019-08-18|
+----------+---------+-----------+



In [0]:
products_filtered_df_1164 = products_df_1164.filter(col("change_date").cast("date") <= "2019-08-16")

In [0]:
window_spec = Window.partitionBy("product_id").orderBy(col("change_date").desc())

In [0]:
latest_prices_df_1164 = products_filtered_df_1164\
                            .withColumn("rank", row_number().over(window_spec)) \
                              .filter(col("rank") == 1) \
                                .select("product_id", col("new_price").alias("price"))

In [0]:
all_product_ids_df_1164 = products_df_1164.select("product_id").distinct()

In [0]:
all_product_ids_df_1164\
    .join(latest_prices_df_1164, on="product_id", how="left") \
        .withColumn("price", coalesce(col("price"), lit(10))) \
            .select("product_id", "price").show()

+----------+-----+
|product_id|price|
+----------+-----+
|         1|   35|
|         2|   50|
|         3|   10|
+----------+-----+

